In [ ]:
# default_exp main

# rfpy - CFRS Rfeye Node Logger binaries parser

In [ ]:
#hide
%load_ext autoreload
%autoreload 2            #Reload the code automatically

In [ ]:
#export
from typing import *
import os
from fastcore.xtras import Path
from fastcore.script import *
from fastcore.utils import parallel, partial
from rfpy.parser import *
import pandas as pd
from rfpy.utils import *
from rich.progress import track, Progress

In [ ]:
#export
def read_meta(filename):
    ext = filename.suffix
    if ext == ".csv":
        df = pd.read_csv(filename)
    elif ext == ".xlsx":
        df = pd.read_excel(filename, engine="openpyxl")
    elif ext == ".fth":
        df = pd.read_feather(filename)
        if "wallclock_datetime" in df.columns:
            df.set_index("wallclock_datetime", inplace=True)
    else:
        raise ValueError(f"Extension {ext} not implemented")
    return df


@call_parse
def process_bin(
    entrada: Param("Diretório contendo arquivos .bin", str),
    saida: Param("Diretório para salvar os arquivos de saída", str),
    recursivo: Param("Buscar arquivos de maneira recursiva?", store_true) = False,
    pastas: Param("Limitar a busca às pastas", Iterable[str]) = None,
    meta: Param("Extrair e Salvar os metadados?", store_true) = False,
    levels: Param("Extrair e Salvar os níveis de Espectro?", store_true) = False,
    ext: Param("Qual extensão salvar os arquivos", str) = ".fth",
    substituir: Param(
        "Reprocessar e substituir arquivos existentes?", store_true
    ) = False,
):

    lista_bins = get_files(
        entrada, extensions=[".bin"], recurse=recursivo, folders=pastas
    )
    parsed_bins = {}
    meta_path = Path(f"{saida}/meta")
    levels_path = Path(f"{saida}/levels")
    meta_path.mkdir(exist_ok=True, parents=True)
    levels_path.mkdir(exist_ok=True, parents=True)

    processed = Path(f"{saida}/processed_log.txt")
    if substituir:
        done = set()
    elif processed.exists():
        done = set(processed.read_text().split("\n"))
    else:
        done = set()

    console.rule("Lista de Arquivos a serem processados", style="bold red")
    console.print(lista_bins, style="bold white", overflow="fold", justify="center")
    if not lista_bins:
        return

    erro_formato = False
    erro_novo = False

    with Progress() as progress:
        bins = progress.track(
            lista_bins,
            total=len(lista_bins),
            description="[green]Processando Arquivos Binários",
        )
        for file in bins:
            progress.description = f"[cyan]Mapeando Blocos do arquivo {file.name}"
            parsed_bins[file.stem] = parse_bin(file, progress=progress)
        if meta:
            lista = [f for f in lista_bins if f.name not in done]
            if not lista:
                erro_novo = True
            else:
                bins = progress.track(
                    lista, total=len(lista), description="[green]Exportando Metadados"
                )
                for file in bins:
                    export_meta(file.stem, parsed_bins[file.stem], meta_path, ext=ext)
                    done.add(file.name)
        if levels:
            if ext == ".csv":
                erro_formato = True
                lista = []
            else:
                lista = [f for f in lista_bins if f.name not in done]
                if not lista:
                    erro_novo = True
                else:
                    bins = progress.track(
                        lista,
                        total=len(lista),
                        description="[green]Exportando Dados de Espectro",
                    )
                    for file in bins:
                        meta_files = get_files(meta_path)
                        for f in meta_files:
                            if file.stem in str(f):
                                meta_file = f
                                break
                        else:
                            console.print(
                                "[red] :poop: Não foram encontrados os arquivos com os metadados, eles serão extraídos primeiro"
                            )
                            export_meta(
                                file.stem, parsed_bins[file.stem], meta_path, ext=ext
                            )
                            done.add(file.name)
                        meta_df = read_meta(meta_file)
                        index = meta_df.index
                        export_level(
                            file.stem,
                            parsed_bins[file.stem],
                            levels_path,
                            ext=ext,
                            index=index,
                        )
                        done.add(file.name)
    if erro_formato:
        console.print(
            ":warning: Dados espectrais em .csv não são suportados pela explosão no tamanho de arquivo :exclamation:"
        )
        console.print("Sugerimos o formato:point_right: [red].fth")
    elif erro_novo:
        console.print(":sleeping: Nenhum arquivo novo a processar :zzz:")
        console.print(
            ":point_up: use --substituir no terminal ou substituir=True na chamada caso queira reprocessar os bins e sobrepôr os arquivos existentes :wink:"
        )
    else:
        processed.write_text("\n".join(sorted(list(done))))
        console.print("kbô :satisfied:")

In [ ]:
blocks = parse_bin('binfiles/rfeye002292_210210_T230245_MaskBroken.bin')
blocks = blocks['blocks']

for k,v in blocks.items():
    print(f'Tipo de Bloco: {k[0]}, Thread_ID: {k[1]}, Nº de blocos {len(v)}, Exemplo de Blocos: {v[0]}')

Tipo de Bloco: 68, Thread_ID: 301, Nº de blocos 6762, Exemplo de Blocos: ((344, 11639), <rfpy.blocks.DType68 object at 0x000001E0DE902F88>)
Tipo de Bloco: 40, Thread_ID: 1, Nº de blocos 723, Exemplo de Blocos: ((23044, 23099), <rfpy.blocks.DType40 object at 0x000001E0B5E97FC8>)
Tipo de Bloco: 68, Thread_ID: 311, Nº de blocos 433, Exemplo de Blocos: ((34292, 35483), <rfpy.blocks.DType68 object at 0x000001E0B6437A08>)
Tipo de Bloco: 68, Thread_ID: 321, Nº de blocos 119, Exemplo de Blocos: ((35488, 36163), <rfpy.blocks.DType68 object at 0x000001E0B627FA88>)
Tipo de Bloco: 68, Thread_ID: 331, Nº de blocos 123, Exemplo de Blocos: ((549428, 549839), <rfpy.blocks.DType68 object at 0x000001E0B5F7DB08>)
Tipo de Bloco: 68, Thread_ID: 371, Nº de blocos 101, Exemplo de Blocos: ((572296, 572635), <rfpy.blocks.DType68 object at 0x000001E0B5F7DF48>)
Tipo de Bloco: 68, Thread_ID: 361, Nº de blocos 16, Exemplo de Blocos: ((2087360, 2087623), <rfpy.blocks.DType68 object at 0x000001E0C26DD708>)
Tipo de B

In [ ]:
getattrs(blocks[(40, 1)][0][1])

{'altitude': 146.1,
 'data_size': 40,
 'gps_datetime': datetime.datetime(2021, 2, 10, 23, 3),
 'gps_status': 'Differential GPS',
 'heading': 0.0,
 'latitude': -12.971591,
 'longitude': -38.48149,
 'num_sattelites': 11,
 'speed': 0.024,
 'thread_id': 1,
 'type': 40,
 'wallclock_datetime': numpy.datetime64('2021-02-10T23:03:00.248904')}

In [ ]:
df = pd.read_feather(r'C:\Users\rsilva\Downloads\saida\meta\rfeye002292_210210_T230245_MaskBroken-B_40_TId_1.fth')

In [ ]:
df2 = pd.read_feather(r'C:\Users\rsilva\Downloads\saida\meta\201201_T154509_MultiplesThreadID-B_63_TId_60.fth')

,wallclock_datetime,start_byte,stop_byte,altitude,data_size,gps_datetime,gps_status,heading,latitude,longitude,num_sattelites,speed,thread_id,type
0,2021-02-10 23:03:00.248904,23044,23099,146.100006,40,2021-02-10 23:03:00,Differential GPS,0.0,-12.971591,-38.481491,11,0.024,1,40
1,2021-02-10 23:04:00.948900,138436,138491,147.100006,40,2021-02-10 23:04:00,Differential GPS,0.0,-12.971593,-38.481491,11,0.020,1,40
2,2021-02-10 23:05:00.832162,251808,251863,147.199997,40,2021-02-10 23:05:00,Differential GPS,0.0,-12.971595,-38.481491,11,0.024,1,40
3,2021-02-10 23:06:00.168887,366372,366427,147.199997,40,2021-02-10 23:06:00,Differential GPS,0.0,-12.971596,-38.481487,11,0.014,1,40
4,2021-02-10 23:07:00.872224,480132,480187,147.500000,40,2021-02-10 23:07:00,Differential GPS,0.0,-12.971601,-38.481487,11,0.016,1,40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
718,2021-02-11 11:01:00.483228,77369048,77369103,147.199997,40,2021-02-11 11:01:00,Differential GPS,0.0,-12.971610,-38.481499,11,0.012,1,40
719,2021-02-11 11:02:00.649141,77445272,77445327,147.899994,40,2021-02-11 11:02:00,Differential GPS,0.0,-12.971608,-38.481499,11,0.018,1,40
720,2021-02-11 11:03:00.947615,77499724,77499779,147.800003,40,2021-02-11 11:03:00,Differential GPS,0.0,-12.971603,-38.481499,11,0.011,1,40
721,2021-02-11 11:04:00.639086,77544700,77544755,146.399994,40,2021-02-11 11:04:00,Differential GPS,0.0,-12.971604,-38.481499,11,0.029,1,40


In [ ]:
df2 = pd.read_feather(r'C:\Users\rsilva\Downloads\saida\meta\201201_T154509_MultiplesThreadID-B_63_TId_60.fth')

In [ ]:
df2.iloc[:, :10]

,wallclock_datetime,start_byte,stop_byte,bw,data_size,gerror,gflags,group_id,id_antenna,n_agc
0,2020-12-01 15:45:09.569136,5276,7427,40,2136,-1,-1,0,0,7
1,2020-12-01 15:45:09.618048,13900,16051,40,2136,-1,-1,0,0,7
2,2020-12-01 15:45:09.670951,20368,22519,40,2136,-1,-1,0,0,7
3,2020-12-01 15:45:09.715272,26836,28987,40,2136,-1,-1,0,0,7
4,2020-12-01 15:45:09.769050,33304,35455,40,2136,-1,-1,0,0,7
...,...,...,...,...,...,...,...,...,...,...
12808,2020-12-01 15:55:52.908790,99968216,99970367,40,2136,-1,-1,0,0,7
12809,2020-12-01 15:55:52.138019,99974684,99976835,40,2136,-1,-1,0,0,7
12810,2020-12-01 15:55:52.191881,99983308,99985459,40,2136,-1,-1,0,0,7
12811,2020-12-01 15:55:52.238091,99991932,99994083,40,2136,-1,-1,0,0,7


In [ ]:
df.num_sattelites.unique()

array([11, 10, 12,  9,  8], dtype=int8)

In [ ]:
# entrada = Path(r'C:\Users\rsilva\Downloads\entrada\pmec')
# saida = Path(r'C:\Users\rsilva\Downloads\saida')
# process_bin(entrada, saida, ext='.xlsx', levels=True)

In [ ]:
from nbdev.export import notebook2script
notebook2script()

Converted 00_main.ipynb.
Converted 01_parser.ipynb.
Converted 02_utils.ipynb.
Converted 03_blocks.ipynb.
Converted 04_constants.ipynb.
Converted index.ipynb.
